# About ConvLSTM
### ConvLSTM을 이용한 도로 구간 속도 예측 기법

#### Dataset
교통데이터는 각 도로링크의 평균 교통
량, 평균 속도, 평균 점유율이며, 기타 도로에 대한
속성 (도로명, 등급, 유형, 제한속도, 제한차량 등)
데이터도 저장된다. 

#### Why ConvLSTM
특정 도로링크의 속도는 이웃하는 도로링크들의 속도에 종속된다. 또한, 해당 도로링크의 이전 시간의 속도들에 의해 현재
속도가 종속된다. 이런 공간적 특징과 시간적 특징을 모두 고려하기 위해 RCNN 기반의 속도 분류기법을 제안하였다. 

In [ ]:
# from keras.models import Sequential
# from keras.layers import Dense, Conv1D, Flatten
# from keras.layers.convolutional_recurrent import ConvLSTM2D

In [25]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler
from torchvision import transforms, utils
from datetime import datetime

In [29]:
X_train = pd.read_csv("/Users/hyunji/1. DaconJeju/feature/train_20221106_1100.csv")
X_test = pd.read_csv("/Users/hyunji/1. DaconJeju/feature/test_20221106_1100.csv")

temp = pd.read_csv('/Users/hyunji/1. DaconJeju/data/raw/train.csv')
temp = temp.query('base_date >= 20220615')[["id","base_date"]]

temptest = pd.read_csv('/Users/hyunji/1. DaconJeju/data/raw/test.csv')
temptest = temptest[["id","base_date"]]

In [30]:
##concat base_date
X_train = pd.merge(X_train,temp, on="id")
X_test = pd.merge(X_test,temptest, on ='id')

##int to datetime
X_train['base_date']= X_train['base_date'].astype('str')
X_train['base_date'] = pd.to_datetime(X_train['base_date'])
X_train.info()

X_test['base_date']= X_test['base_date'].astype('str')
X_test['base_date'] = pd.to_datetime(X_test['base_date'])
X_test.info()

#y_train
y_train = X_train[['target',"base_date"]].set_index("base_date")
# y_train.reset_index(inplace=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 524074 entries, 0 to 524073
Data columns (total 47 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   id                     524074 non-null  object        
 1   target                 524074 non-null  float64       
 2   day_of_week            524074 non-null  int64         
 3   base_hour              524074 non-null  float64       
 4   lane_count             524074 non-null  float64       
 5   road_rating            524074 non-null  float64       
 6   road_name              524074 non-null  int64         
 7   maximum_speed_limit    524074 non-null  float64       
 8   weight_restricted      524074 non-null  float64       
 9   road_type              524074 non-null  float64       
 10  start_turn_restricted  524074 non-null  int64         
 11  end_turn_restricted    524074 non-null  int64         
 12  length                 524074 non-null  floa

In [33]:
X_train.drop(columns = ["id","target"], inplace=True)
X_train=X_train.set_index("base_date")
X_train.head()

,day_of_week,base_hour,lane_count,road_rating,road_name,maximum_speed_limit,weight_restricted,road_type,start_turn_restricted,end_turn_restricted,...,msl_std,coor_std,edge_std,rn_frequency,hour_frequency,dow_frequency,start_frequency,end_frequency,coor_frequency,edge_frequency
base_date,,,,,,,,,,,,,,,,,,,,,
2022-06-23,1,0.807704,-1.321033,0.387314,45,-0.074194,2.722643,1.958631,0,0,...,0.896351,-0.219310,-0.301486,-0.889581,0.532415,-0.612720,-0.864259,-0.860314,-1.163263,-0.736359
2022-07-28,1,1.318952,0.168554,-1.132946,33,-0.074194,-0.367277,-0.510561,1,0,...,0.896351,0.357531,0.284045,0.154787,0.285837,-0.612720,-0.668633,-0.676695,0.666522,0.220716
2022-06-23,1,-0.015124,-1.321033,-1.132946,37,-0.074194,-0.367277,-0.510561,0,0,...,0.896351,-1.278157,-1.383956,-0.567830,0.551844,-0.612720,-0.323022,-0.688626,0.610618,0.202501
2022-07-24,4,-1.499524,-1.321033,1.040124,44,-0.897766,-0.367277,-0.510561,0,0,...,-1.174040,-0.556979,-0.645615,-1.060430,-0.963220,1.122858,1.074828,1.613966,0.459446,0.151467
2022-07-01,0,1.443150,0.168554,1.040124,28,-0.897766,-0.367277,-0.510561,0,0,...,-1.174040,0.874309,0.806080,-1.680418,-0.922693,-1.058748,-0.664666,-0.672732,0.685332,0.226770


In [42]:
X_train.reset_index().sort_values(by = "base_date").set_index("base_date")
y_train.reset_index().sort_values(by = "base_date").set_index("base_date")

print(X_train.head())
print(y_train.head())


            day_of_week  base_hour  lane_count  road_rating  road_name  \
base_date                                                                
2022-06-23            1   0.807704   -1.321033     0.387314         45   
2022-07-28            1   1.318952    0.168554    -1.132946         33   
2022-06-23            1  -0.015124   -1.321033    -1.132946         37   
2022-07-24            4  -1.499524   -1.321033     1.040124         44   
2022-07-01            0   1.443150    0.168554     1.040124         28   

            maximum_speed_limit  weight_restricted  road_type  \
base_date                                                       
2022-06-23            -0.074194           2.722643   1.958631   
2022-07-28            -0.074194          -0.367277  -0.510561   
2022-06-23            -0.074194          -0.367277  -0.510561   
2022-07-24            -0.897766          -0.367277  -0.510561   
2022-07-01            -0.897766          -0.367277  -0.510561   

            start_turn_re

In [47]:
X_test = X_test.drop(columns = ["id"]).set_index("base_date").sort_values(by = "base_date")

In [48]:
# model input data size
print('X_train.shape:', X_train.shape, 'X_test.shape', X_test.shape)
print('feature:', X_train.columns)

X_train.shape: (524074, 44) X_test.shape (291241, 44)
feature: Index(['day_of_week', 'base_hour', 'lane_count', 'road_rating', 'road_name',
       'maximum_speed_limit', 'weight_restricted', 'road_type',
       'start_turn_restricted', 'end_turn_restricted', 'length', 'speed',
       'time', 'start_cluster', 'end_cluster', 'start_node_label',
       'end_node_label', 'edge_label', 'coor_label', 'base_month', 'base_day',
       'base_no_week', 'pca_lat', 'pca_long', 'harversine', 'bearing',
       'rn_target', 'hour_target', 'dow_target', 'msl_target', 'edge_target',
       'rn_std', 'hour_std', 'dow_std', 'msl_std', 'coor_std', 'edge_std',
       'rn_frequency', 'hour_frequency', 'dow_frequency', 'start_frequency',
       'end_frequency', 'coor_frequency', 'edge_frequency'],
      dtype='object')


# ConvLSTM Model

### 3D Transforms

In [49]:
## X / Y Split
X_trainL, y_trainL = [], []
sequence = 10
for index in range(len(X_train) - sequence):
    X_trainL.append(np.array(X_train[index: index + sequence]))
    y_trainL.append(np.ravel(y_train[index + sequence:index + sequence + 1]))
X_train, y_train = np.array(X_trainL), np.array(y_trainL)

## Retype and Reshape
X_train = X_train.reshape(X_train.shape[0], sequence, -1)
# X_test = X_test.reshape(X_test.shape[0], sequence, -1)
print('X_train:', X_train.shape, 'y_train:', y_train.shape)
# print('X_test:', X_test.shape, 'Y_test:', Y_test.shape)

X_train: (524064, 10, 44) y_train: (524064, 1)


In [53]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(X_train, y_train, test_size=0.2, shuffle=False, random_state=2022)

In [54]:
#Tensor 형태로 변환
def make_Tensor(array):
    return torch.from_numpy(array).float()

X_train = make_Tensor(train_X)
y_train = make_Tensor(train_y)
X_val = make_Tensor(val_X)
y_val = make_Tensor(val_y)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv1D
# from keras.layers.convolutional_recurrent import ConvLSTM2D

In [ ]:
model = Sequential()

# Input shape: (samples, time, channels, rows, cols) see: https://keras.io/layers/recurrent/#convlstm2d
model.add(ConvLSTM1D(
        filters=32,
        kernel_size=2 ,
        input_shape=(None, 7,1),
        padding='same',
        return_sequences=True))
model.add(layers.Activation(activations.relu))
model.add(BatchNormalization())
model.add(Flatten())

model.add(Dense(64, kernel_initializer='TruncatedNormal'))
model.add(layers.Activation(activations.relu))
model.add(Dropout(0.2))

model.add(Dense(32, kernel_initializer='TruncatedNormal'))
model.add(layers.Activation(activations.relu))
model.add(Dropout(0.2))

model.add(Dense(1))a

In [56]:
class ConvLSTM(nn.Module):
    def __init__(self, n_features, n_hidden, seq_len, n_layers):
        super(ConvLSTM, self).__init__()
        self.n_hidden = n_hidden
        self.seq_len = seq_len
        self.n_layers = n_layers
        self.c1 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size = 2, stride = 1) # 1D CNN 레이어 추가
        self.lstm = nn.LSTM(
            input_size=n_features,
            hidden_size=n_hidden,
            num_layers=n_layers
        )
        self.linear = nn.Linear(in_features=n_hidden, out_features=1)
    def reset_hidden_state(self):
        self.hidden = (
            torch.zeros(self.n_layers, self.seq_len-1, self.n_hidden),
            torch.zeros(self.n_layers, self.seq_len-1, self.n_hidden)
        )
    def forward(self, sequences):
        sequences = self.c1(sequences.view(len(sequences), 1, -1))
        lstm_out, self.hidden = self.lstm(
            sequences.view(len(sequences), self.seq_len-1, -1),
            self.hidden
        )
        last_time_step = lstm_out.view(self.seq_len-1, len(sequences), self.n_hidden)[-1]
        y_pred = self.linear(last_time_step)
        return y_pred

In [60]:
def train_model(model, train_data, train_labels, val_data=None, val_labels=None, num_epochs=100, verbose = 10, patience = 10):
    loss_fn = torch.nn.L1Loss() #
    optimiser = torch.optim.Adam(model.parameters(), lr=0.001)
    train_hist = []
    val_hist = []
    for t in range(num_epochs):

        epoch_loss = 0

        for idx, seq in enumerate(train_data): # sample 별 hidden state reset을 해줘야 함 

            model.reset_hidden_state()

            # train loss
            seq = torch.unsqueeze(seq, 0)
            y_pred = model(seq)
            loss = loss_fn(y_pred[0].float(), train_labels[idx]) # 1개의 step에 대한 loss

            # update weights
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()

            epoch_loss += loss.item()

        train_hist.append(epoch_loss / len(train_data))

        if val_data is not None:

            with torch.no_grad():

                val_loss = 0

                for val_idx, val_seq in enumerate(val_data):

                    model.reset_hidden_state() #seq 별로 hidden state 초기화 

                    val_seq = torch.unsqueeze(val_seq, 0)
                    y_val_pred = model(val_seq)
                    val_step_loss = loss_fn(y_val_pred[0].float(), val_labels[val_idx])

                    val_loss += val_step_loss
                
            val_hist.append(val_loss / len(val_data)) # val hist에 추가

            ## verbose 번째 마다 loss 출력 
            if t % verbose == 0:
                print(f'Epoch {t} train loss: {epoch_loss / len(train_data)} val loss: {val_loss / len(val_data)}')

            ## patience 번째 마다 early stopping 여부 확인
            if (t % patience == 0) & (t != 0):
                
                ## loss가 커졌다면 early stop
                if val_hist[t - patience] < val_hist[t] :

                    print('\n Early Stopping')

                    break

        elif t % verbose == 0:
            print(f'Epoch {t} train loss: {epoch_loss / len(train_data)}')

            
    return model, train_hist, val_hist

In [61]:
model = ConvLSTM(
    n_features=1,
    n_hidden=4,
    seq_len=sequence,
    n_layers=1
)

In [62]:
model, train_hist, val_hist = train_model(
    model,
    X_train,
    y_train,
    X_val,
    y_val,
    num_epochs=100,
    verbose=10,
    patience=50
)

tensor([[[ 1.0000e+00,  8.0770e-01, -1.3210e+00,  3.8731e-01,  4.5000e+01,
          -7.4194e-02,  2.7226e+00,  1.9586e+00,  0.0000e+00,  0.0000e+00,
          -1.8045e+00, -1.8335e+00, -1.9935e+00,  1.6478e+00,  1.6478e+00,
          -2.1947e+00, -2.1964e+00, -2.1803e+00, -5.1290e-03, -1.0495e+00,
           4.3797e-01,  2.7657e-01,  5.2638e-01,  1.2056e+00, -1.8283e+00,
          -8.2024e-01,  1.1353e+00, -3.0140e-13,  1.9544e-01,  4.2496e-01,
           8.3176e-01, -4.8232e-01,  4.1242e-13,  1.2901e+00,  8.9635e-01,
          -2.1931e-01, -3.0149e-01, -8.8958e-01,  5.3241e-01, -6.1272e-01,
          -8.6426e-01, -8.6031e-01, -1.1633e+00, -7.3636e-01],
         [ 1.0000e+00,  1.3190e+00,  1.6855e-01, -1.1329e+00,  3.3000e+01,
          -7.4194e-02, -3.6728e-01, -5.1056e-01,  1.0000e+00,  0.0000e+00,
           6.0968e-01, -8.6488e-01,  5.3375e-01,  4.7593e-01,  4.7593e-01,
          -2.1552e+00,  7.3935e-01, -2.1538e+00, -2.3484e-03,  9.5281e-01,
           1.1901e+00,  1.3580e+00,  

RuntimeError: shape '[1, 9, -1]' is invalid for input of size 439